# Recursive data types and functions

## Recursive types

### The `List` type

In [ ]:
// type List[T] = 1 + T*List[T]
//              = 1 + T*(1+T*List[T]) = 1 + T*1 + T*T*List[T]
//              ...
//              = 1 + T + T*T + T*T*T + ....

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [4]:
object StdDefinition {
    // type List[T] = 1 + T * List[T]
    sealed abstract class List[T]
    case class Nil[T]() extends List[T]
    case class NonEmpty[T](head: T, tail: List[T]) 
        extends List[T]
    
    
    // [1, 2, 3]
    val l: List[Int] = 
        NonEmpty(1, NonEmpty(2, NonEmpty(3, Nil())))
}

defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [5]:
object ActualStdDefinition{
    sealed abstract class List[+T]
    case object Nil extends List[Nothing]
    case class ::[T](head: T, tail: List[T]) 
        extends List[T]
}

defined object ActualStdDefinition

In [6]:
val l: List[Int] = ::(1, ::(2, ::(3, Nil)))

l: List[Int] = List(1, 2, 3)

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [9]:
// Less beautifully 


// More idiomatically
val l: List[Int] = 1 :: (2 :: (3 :: Nil))
val l2: List[Int] = List(1,2,3)

l: List[Int] = List(1, 2, 3)
l2: List[Int] = List(1, 2, 3)

And can also pattern match lists, similarly:

In [13]:
// Less beautifully

List(1,2,3,4) match {
    case Nil => "vacía"
    case ::(head, tail) => "no vacía con head = " + head
}
// more idiomatically

List(1,2,3,4) match {
    case Nil => "vacía"
    case head :: tail => "blablablá"
}
// or


res12_0: String = "no vac\u00eda con head = 1"
res12_1: String = "blablabl\u00e1"

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [14]:
List(1,2,3).length

res13: Int = 3

In [43]:
// Using mutable variables

def lengthI[A](l: List[A]): Int = {
    var out: Int = 0
    for (e <- l)
        out = out + 1
    out
}

defined function lengthI

In [44]:
// Using mutable variables

def lengthTR[A](list: List[A]): Int = {
    
    def aux(out: Int, l: List[A]): Int = 
        l match {
            case Nil => out
            case head :: tail => 
                aux(out + 1, tail)
        }
    
    aux(0, list)
}
/* 
    var out: Int = 0
    for (e <- l)
        out = out + 1
    out
}*/

defined function lengthTR

In [45]:
lengthI(List(1,2,3))

res44: Int = 3

The recursive function is implemented as follows: 

In [46]:
// Using recursive functions

def lengthR[A](l: List[A]): Int = 
    l match {
        case Nil => 
            0 : Int
        case head :: tail =>
            val tailLength: Int = lengthR(tail)
            tailLength + 1 : Int 
    }

defined function lengthR

In [47]:
lengthR(List(1,2,3))

res46: Int = 3

In [48]:
val biglist = List.fill(10000)(1) // 1 :: (1 :: (1 :: ...))

biglist: List[Int] = List(
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
...

In [49]:
lengthTR(biglist)

res48: Int = 10000

In [50]:
lengthR(biglist)

: 

In [ ]:
// Using recursive functions

def lengthR[A](l: List[A]): Int = 
    l match {
        case Nil => 
            0
        case head :: tail =>
            lengthR(tail) + 1
    }

In [51]:
// Using recursive functions

def dyv[A, B](l: List[A])(atomic: B, 
                          compose: (B, A) => B): B = 
    l match {
        case Nil => 
             atomic : B
        case head :: tail =>
            val tailSol: B = dyv(tail)(atomic, compose)
            compose(tailSol, head) : B 
    }

defined function dyv

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [51]:
// Using tail-recursive functions



We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [51]:
// First, imperatively



In [51]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [51]:
// Imperatively


In [51]:
// Tail-recursive


In [51]:
// Plain recursive


### Unit testing with `scalatest`

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [53]:
class LengthSpec extends FlatSpec with Matchers{
    
    "length of empty" should "work" in { 
        lengthR(List()) shouldBe 0
    }
    
    "length of non-empty lists" should "work" in {
        lengthR(List(1)) shouldBe 1
        lengthR(List(1,2,3,4)) shouldBe 4
    }
}


defined class LengthSpec

In [54]:
class LengthSpec(lengthF: List[Int] => Int) 
extends FlatSpec with Matchers{
    
    "length of empty" should "work" in { 
        lengthF(List()) shouldBe 0
    }
    
    "length of non-empty lists" should "work" in {
        lengthF(List(1)) shouldBe 1
        lengthF(List(1,2,3,4)) shouldBe 4
    }
}


defined class LengthSpec

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [56]:
run(new LengthSpec(lengthTR[Int]))
run(new LengthSpec((l: List[Int]) => lengthR(l)))
run(new LengthSpec(lengthI))

cmd53$Helper$LengthSpec:
length of empty
- should work
length of non-empty lists
- should work
cmd53$Helper$LengthSpec:
length of empty
- should work
length of non-empty lists
- should work
cmd53$Helper$LengthSpec:
length of empty
- should work
length of non-empty lists
- should work


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [58]:
class TestSum(sum: List[Int] => Int) 
extends FlatSpec with Matchers{
    "sum" should "work" in {
        sum(List()) shouldBe 0
        sum(List(1)) shouldBe 1
        sum(List(5)) shouldBe 5
        sum(List(2,2,34,5)) shouldBe 43
    }
}

defined class TestSum

In [60]:
// Recursively
def sumR(l: List[Int]): Int = 
    l match {
        case Nil => 
            0 : Int
        case head :: tail => 
            val tailSum: Int = sumR(tail)
            tailSum + head : Int
    }


defined function sumR

In [61]:
run(new TestSum(sumR))

cmd57$Helper$TestSum:
sum
- should work


In [62]:
// for-loops

def sumI(l: List[Int]): Int = {
    var out: Int = 0
    for (e <- l)
        out = out + e
    out
}

defined function sumI

In [63]:
run(new TestSum(sumI))

cmd57$Helper$TestSum:
sum
- should work


In [70]:
// With tail-recursion

def sumTR(list: List[Int]): Int = {
    
    def aux(out: Int, l: List[Int]): Int = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(out + e, tail)
        }
    
    aux(0, list)
    /*
    var out: Int = 0
    for (e <- l)
        out = out + e
    out*/
}

defined function sumTR

In [69]:
run(new TestSum(sumTR))

cmd57$Helper$TestSum:
sum
- should work *** FAILED ***
  0 was not equal to 1 (cmd57.sc:5)


### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [78]:
class TestProduct(product: List[Int] => Int) 
extends FlatSpec with Matchers{
    "product" should "work" in {
        product(List()) shouldBe 1
        product(List(4)) shouldBe 4
        product(List(1,2,3,4)) shouldBe 24
        product(List(1,0,3,4)) shouldBe 0
        // List(1,2,3,.....,0,4,5,5,6,7,.....)
    }
}

defined class TestProduct

This is the common recursive implementation:

In [79]:
def productR(l: List[Int]): Int = 
    l match {
        case Nil => 
            1 : Int
        case head :: tail => 
            val tailProduct: Int = productR(tail)
            tailProduct * head : Int
    }


defined function productR

In [80]:
def productROpt(l: List[Int]): Int = 
    l match {
        case Nil => 
            1 : Int
        case head :: tail => 
            if (head == 0) 0
            else productROpt(tail) * head
    }


defined function productROpt

In [80]:
def productROpt(l: List[Int]): Int = 
    l match {
        case Nil => 
            1 : Int
        case head :: tail if (head == 0) => 
            0
        case head :: tail => 
            productROpt(tail) * head
    }


defined function productROpt

In [82]:
def productROpt(l: List[Int]): Int = 
    l match {
        case Nil => 
            1 : Int
        case 0 :: tail => 
            0
        case head :: tail => 
            productROpt(tail) * head
    }


defined function productROpt

It works as expected: 

In [81]:
run(new TestProduct(productR))
run(new TestProduct(productROpt))

cmd77$Helper$TestProduct:
product
- should work
cmd77$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
/*def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => head * product(tail)
    }
    */

In [83]:
def productTR(list: List[Int]): Int = {
    def aux(out: Int, l: List[Int]): Int = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(out * e, tail)
        }
    
    aux(1, list)
}

defined function productTR

In [84]:
def productTROpt(list: List[Int]): Int = {
    def aux(out: Int, l: List[Int]): Int = 
        l match {
            case Nil => out
            case 0 :: tail => 0
            case e :: tail => 
                aux(out * e, tail)
        }
    
    aux(1, list)
}

defined function productTROpt

In [85]:
run(new TestProduct(productTROpt))

cmd77$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [87]:
class TestMember(member: (List[Int], Int) => Boolean)
extends FlatSpec with Matchers{
    "member" should "work" in {
        member(List(1,2,3), 1) shouldBe true
        member(List(1,2,3), 2) shouldBe true
        member(List(1,2,3), 4) shouldBe false
        member(List(), 3) shouldBe false
    }
}

defined class TestMember

In [89]:
def memberR[A](l: List[A], e: A): Boolean = 
    l match {
        case Nil => 
            false
        case head :: tail => 
            val tailMember: Boolean = memberR(tail, e)
            (head == e) || tailMember
    }

defined function memberR

In [91]:
def memberR[A](l: List[A], e: A): Boolean = 
    l match {
        case Nil => 
            false
        case head :: tail => 
            if (head == e) true
            else {
                val tailMember: Boolean = memberR(tail, e)
                tailMember
            }
    }

defined function memberR

In [93]:
def memberR[A](l: List[A], e: A): Boolean = 
    l match {
        case Nil => false
        case head :: tail if head == e => true
        case head :: tail => memberR(tail, e)
    }

defined function memberR

In [98]:
{ val e = 1 ; 
    { val e = 2
        println(e)
    }
}

2


e: Int = 1

In [99]:
def memberR[A](l: List[A], e: A): Boolean = 
    l match {
        case Nil => false
        case `e` :: tail => true
        case head :: tail => memberR(tail, e)
    }

defined function memberR

In [100]:
run(new TestMember(memberR))

cmd86$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [102]:
class TestLast(last: List[Int] => Option[Int]) 
extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1,2)) shouldBe Some(2)
        last(List(1)) shouldBe Some(1)
        last(Nil) shouldBe None
    }
}

defined class TestLast

In [107]:
def lastR(l: List[Int]): Option[Int] = 
    l match {
        case Nil => 
            None
        case head :: tail =>
            if (tail == Nil) Some(head)
            else {
                val tailLast: Option[Int] = lastR(tail)
                tailLast
            }
    }

defined function lastR

In [111]:
def lastR(l: List[Int]): Option[Int] = 
    l match {
        case Nil => None
        case head :: Nil => Some(head)
        case _ :: tail => lastR(tail)
    }

defined function lastR

In [112]:
run(new TestLast(lastR))

cmd101$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [ ]:
object TestInsertLast extends FlatSpec with Matchers{
    "insertLast" should "work" in {
    }
}

In [ ]:
run(TestInsertLast)

### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [5]:
List(1,2,3) :+ 4

res4: List[Int] = List(1, 2, 3, 4)

In [6]:
class TestReverse(reverse: List[Int] => List[Int]) extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(1::List(2,3,4)) shouldBe List(4,3,2) :+ 1
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
        reverse(List()) shouldBe List()
    }
}

defined class TestReverse

In [8]:
// Recursively: Really inefficient 
def reverseR(l: List[Int]): List[Int] =  
    l match {
        case Nil => List() : List[Int]
        case head :: tail => 
            val tailReverse: List[Int] = reverseR(tail) 
            tailReverse :+ head : List[Int] 
    }

defined function reverseR

In [9]:
run(new TestReverse(reverseR))

cmd5$Helper$TestReverse:
reverse
- should work


In [10]:
// Tail-recursive, efficiently
def reverseTR(list: List[Int]): List[Int] = {
    def aux(out: List[Int], l: List[Int]): List[Int] = 
        l match {
            case Nil => out
            case head :: tail => 
                aux(head :: out, tail)
        }

    aux(List(), list)
}

defined function reverseTR

In [11]:
run(new TestReverse(reverseTR))

cmd5$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)